## Practical 1: word2vec
<p>Oxford CS - Deep NLP 2017<br>
https://www.cs.ox.ac.uk/teaching/courses/2016-2017/dl/</p>
<p>[Yannis Assael, Brendan Shillingford, Chris Dyer]</p>

This practical is presented as an IPython Notebook, with the code written for recent versions of **Python 3**. The code in this practical will not work with Python 2 unless you modify it. If you are using your own Python installation, ensure you have a setup identical to that described in the installation shell script (which is intended for use with the department lab machines). We will be unable to support installation on personal machines due to time constraints, so please use the lab machines and the setup script if you are unfamiliar with how to install Anaconda.

To execute a notebook cell, press `shift-enter`. The return value of the last command will be displayed, if it is not `None`.

Potentially useful library documentation, references, and resources:

* IPython notebooks: <https://ipython.org/ipython-doc/3/notebook/notebook.html#introduction>
* Numpy numerical array library: <https://docs.scipy.org/doc/>
* Gensim's word2vec: <https://radimrehurek.com/gensim/models/word2vec.html>
* Bokeh interactive plots: <http://bokeh.pydata.org/en/latest/> (we provide plotting code here, but click the thumbnails for more examples to copy-paste)
* scikit-learn ML library (aka `sklearn`): <http://scikit-learn.org/stable/documentation.html>
* nltk NLP toolkit: <http://www.nltk.org/>
* tutorial for processing xml in python using `lxml`: <http://lxml.de/tutorial.html> (we did this for you below, but in case you need it in the future)

In [1]:
import numpy as np
import os
from random import shuffle
import re

In [2]:
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

### Part 0: Download the TED dataset

In [3]:
import urllib.request
import zipfile
import lxml.etree

In [4]:
# Download the dataset if it's not already there: this may take a minute as it is 75MB
if not os.path.isfile('ted_en-20160408.zip'):
    urllib.request.urlretrieve("https://wit3.fbk.eu/get.php?path=XML_releases/xml/ted_en-20160408.zip&filename=ted_en-20160408.zip", filename="ted_en-20160408.zip")

In [5]:
# For now, we're only interested in the subtitle text, so let's extract that from the XML:
with zipfile.ZipFile('ted_en-20160408.zip', 'r') as z:
    doc = lxml.etree.parse(z.open('ted_en-20160408.xml', 'r'))
input_text = '\n'.join(doc.xpath('//content/text()'))
del doc

### Part 1: Preprocessing

In this part, we attempt to clean up the raw subtitles a bit, so that we get only sentences. The following substring shows examples of what we're trying to get rid of. Since it's hard to define precisely what we want to get rid of, we'll just use some simple heuristics.

In [6]:
i = input_text.find("Hyowon Gweon: See this?")
input_text[i-20:i+150]

' baby does.\n(Video) Hyowon Gweon: See this? (Ball squeaks) Did you see that? (Ball squeaks) Cool. See this one? (Ball squeaks) Wow.\nLaura Schulz: Told you. (Laughs)\n(Vide'

Let's start by removing all parenthesized strings using a regex:

In [7]:
input_text_noparens = re.sub(r'\([^)]*\)', '', input_text)

We can verify the same location in the text is now clean as follows. We won't worry about the irregular spaces since we'll later split the text into sentences and tokenize it anyway.

In [8]:
i = input_text_noparens.find("Hyowon Gweon: See this?")
input_text_noparens[i-20:i+150]

"hat the baby does.\n Hyowon Gweon: See this?  Did you see that?  Cool. See this one?  Wow.\nLaura Schulz: Told you. \n HG: See this one?  Hey Clara, this one's for you. You "

Now, let's attempt to remove speakers' names that occur at the beginning of a line, by deleting pieces of the form "`<up to 20 characters>:`", as shown in this example. Of course, this is an imperfect heuristic. 

In [9]:
sentences_strings_ted = []
for line in input_text_noparens.split('\n'):
    m = re.match(r'^(?:(?P<precolon>[^:]{,20}):)?(?P<postcolon>.*)$', line)
    sentences_strings_ted.extend(sent for sent in m.groupdict()['postcolon'].split('.') if sent)

# Uncomment if you need to save some RAM: these strings are about 50MB.
# del input_text, input_text_noparens

# Let's view the first few:
sentences_strings_ted[:5]

["Here are two reasons companies fail: they only do more of the same, or they only do what's new",
 'To me the real, real solution to quality growth is figuring out the balance between two activities: exploration and exploitation',
 ' Both are necessary, but it can be too much of a good thing',
 'Consider Facit',
 " I'm actually old enough to remember them"]

Now that we have sentences, we're ready to tokenize each of them into words. This tokenization is imperfect, of course. For instance, how many tokens is "can't", and where/how do we split it? We'll take the simplest naive approach of splitting on spaces. Before splitting, we remove non-alphanumeric characters, such as punctuation. You may want to consider the following question: why do we replace these characters with spaces rather than deleting them? Think of a case where this yields a different answer.

In [10]:
sentences_ted = []
for sent_str in sentences_strings_ted:
    tokens = re.sub(r"[^a-z0-9]+", " ", sent_str.lower()).split()
    sentences_ted.append(tokens)

Two sample processed sentences:

In [11]:
len(sentences_ted)

266694

In [12]:
print(sentences_ted[0])
print(sentences_ted[1])

['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new']
['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation']


### Part 2: Word Frequencies

If you store the counts of the top 1000 words in a list called `counts_ted_top1000`, the code below will plot the histogram requested in the writeup.

In [83]:
freq_words = {}
for sentence in sentences_ted:
    for i in sentence:
        if i in freq_words:
            freq_words[i] += 1
        else:
            freq_words[i] = 1
keys = sorted(freq_words, key=lambda x: int(freq_words[x]), reverse=True)
keys = keys[:1000]

counts_ted_top1000=[]

for i in keys[:1000]:
    counts_ted_top1000.append(freq_words[i])
words_top_ted = sorted(freq_words, reverse=True, key=freq_words.__getitem__)



Plot distribution of top-1000 words

In [14]:
hist, edges = np.histogram(counts_ted_top1000, density=True, bins=100, normed=True)

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="Top-1000 words distribution")
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="#555555")
show(p)

### Part 3: Train Word2Vec

In [17]:
import gensim
from gensim.models import Word2Vec

In [18]:
model_ted = Word2Vec(sentences_ted, size=100, window=5, min_count=5, workers=4)
model_ted.save('/tmp/w2v_model_ted')

### Part 4: Ted Learnt Representations

Finding similar words: (see gensim docs for more functionality of `most_similar`)

In [19]:
model_ted.most_similar("man")

[('woman', 0.8480091094970703),
 ('guy', 0.8013213872909546),
 ('girl', 0.7417789697647095),
 ('boy', 0.7409495115280151),
 ('lady', 0.733483612537384),
 ('soldier', 0.7245442867279053),
 ('kid', 0.7161077260971069),
 ('gentleman', 0.7066822052001953),
 ('writer', 0.68292635679245),
 ('poet', 0.6703674793243408)]

In [20]:
model_ted.most_similar("computer")

[('machine', 0.739233136177063),
 ('software', 0.7268202304840088),
 ('3d', 0.6859555840492249),
 ('device', 0.6841541528701782),
 ('robot', 0.6589328646659851),
 ('video', 0.643180787563324),
 ('program', 0.6373674869537354),
 ('mechanical', 0.6360824704170227),
 ('interface', 0.6360324025154114),
 ('camera', 0.6288957595825195)]

In [21]:
model_ted.most_similar("brain")

[('body', 0.7546346783638),
 ('activity', 0.719536542892456),
 ('memory', 0.714745819568634),
 ('cortex', 0.6921871900558472),
 ('pattern', 0.684168815612793),
 ('dna', 0.6518138647079468),
 ('skin', 0.6397645473480225),
 ('tumor', 0.6337646842002869),
 ('structure', 0.6280688047409058),
 ('cells', 0.6229146718978882)]

In [22]:
model_ted.most_similar(positive=['french', 'english'], negative=['spanish'])

[('american', 0.5304312705993652),
 ('poetry', 0.5116821527481079),
 ('chinese', 0.5025721788406372),
 ('writing', 0.4998958110809326),
 ('author', 0.4958054721355438),
 ('obama', 0.48391735553741455),
 ('feynman', 0.4779726266860962),
 ('word', 0.46899664402008057),
 ('teacher', 0.4665669798851013),
 ('art', 0.4664309620857239)]

#### t-SNE visualization
To use the t-SNE code below, first put a list of the top 1000 words (as strings) into a variable `words_top_ted`. The following code gets the corresponding vectors from the model, assuming it's called `model_ted`:

In [23]:
# This assumes words_top_ted is a list of strings, the top 1000 words
words_top_vec_ted = model_ted[words_top_ted[:1000]]

In [24]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, random_state=0)
words_top_ted_tsne = tsne.fit_transform(words_top_vec_ted)

In [25]:
p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=words_top_ted_tsne[:,0],
                                    x2=words_top_ted_tsne[:,1],
                                    names=words_top_ted))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

/Users/galiasadykova/anaconda/lib/python3.6/site-packages/bokeh/models/sources.py:89: BokehUserWarning: ColumnDataSource's columns must be of the same length
  lambda: warnings.warn("ColumnDataSource's columns must be of the same length", BokehUserWarning))


### Part 5: Wiki Learnt Representations

Download dataset

In [57]:
input_text = str(open('wiki.train.raw', 'r').read())
print(input_text[:1000])

 
 = Valkyria Chronicles III = 
 
 Senjō no Valkyria 3 : Unrecorded Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . Employing the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs parallel to the first game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " Calamaty Raven " . 
 The game began development in 2010 , carrying over a large portion of the work done on Valkyria Chronicles II . While it retained the standard features of the series , it also underwent multiple adjustments , such as making the game more forgiving for series 

Preprocess sentences (note that it's important to remove small sentences for performance)

In [58]:
sentences_wiki = []
for line in input_text.split('\n'):
    s = [x for x in line.split('.') if x and len(x.split()) >= 5]
    sentences_wiki.extend(s)
    
for s_i in range(len(sentences_wiki)):
    sentences_wiki[s_i] = re.sub("[^a-z]", " ", sentences_wiki[s_i].lower())
    sentences_wiki[s_i] = re.sub(r'\([^)]*\)', '', sentences_wiki[s_i])
del input_text

In [59]:
# sample 1/5 of the data
shuffle(sentences_wiki)
print(len(sentences_wiki))
sentences_wiki = sentences_wiki[:int(len(sentences_wiki)/5)]
print(len(sentences_wiki))
sentences_wiki = [sentence.split() for sentence in sentences_wiki]

1429102
285820


Now, repeat all the same steps that you performed above. You should be able to reuse essentially all the code.

In [88]:
words_wiki = {}
for sentence in sentences_wiki:
    for i in sentence:
        if i in words_wiki:
            words_wiki[i] += 1
        else:
            words_wiki[i] = 1


words_top_wiki = sorted(words_wiki, reverse=True, key=words_wiki.__getitem__)




In [89]:

keys = sorted(words_wiki, key=lambda x: int(words_wiki[x]), reverse=True)
keys = keys[:1000]

counts_ted_top1000=[]

for i in keys[:1000]:
    counts_ted_top1000.append(words_wiki[i])




In [90]:
hist, edges = np.histogram(counts_ted_top1000, density=True, bins=100, normed=True)

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="Top-1000 words distribution")
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="#555555")
show(p)

In [91]:
model_wiki = Word2Vec(sentences_wiki, size=100, window=5, min_count=5, workers=4)
model_wiki.save('/tmp/w2v_model_wiki')

In [92]:
#Wiki Learnt Representations
model_wiki.most_similar("man")

[('woman', 0.7285497784614563),
 ('girl', 0.6863499879837036),
 ('person', 0.6637763977050781),
 ('boy', 0.6599622964859009),
 ('sword', 0.6168099641799927),
 ('dog', 0.6038172245025635),
 ('child', 0.594602108001709),
 ('spider', 0.5841220021247864),
 ('warrior', 0.5798591375350952),
 ('figure', 0.5749272108078003)]

In [93]:
model_wiki.most_similar("computer")

[('software', 0.8112918734550476),
 ('graphics', 0.8102688789367676),
 ('multiplayer', 0.7926225662231445),
 ('user', 0.7785351872444153),
 ('arcade', 0.7630363702774048),
 ('gameplay', 0.7612342834472656),
 ('visual', 0.7603391408920288),
 ('hardware', 0.7542281150817871),
 ('programming', 0.7534397840499878),
 ('virtual', 0.7517393231391907)]

In [94]:
model_wiki.most_similar("brain")

[('behavior', 0.8336255550384521),
 ('symptoms', 0.8183766603469849),
 ('disorder', 0.8172847032546997),
 ('animal', 0.8155925273895264),
 ('diet', 0.8142272233963013),
 ('genetic', 0.8043161034584045),
 ('liver', 0.788902759552002),
 ('disease', 0.7884811162948608),
 ('spectrum', 0.7844111323356628),
 ('dna', 0.7841495871543884)]

In [95]:
model_wiki.most_similar(positive=['french', 'english'], negative=['spanish'])

[('language', 0.651555597782135),
 ('literature', 0.6064457297325134),
 ('greek', 0.5759106874465942),
 ('tradition', 0.5652124881744385),
 ('scottish', 0.561388373374939),
 ('welsh', 0.5587080717086792),
 ('naturalist', 0.540568470954895),
 ('christianity', 0.5365082025527954),
 ('irish', 0.5362892150878906),
 ('kannada', 0.5268388390541077)]

#### t-SNE visualization

In [96]:
# This assumes words_top_wiki is a list of strings, the top 1000 words
words_top_vec_wiki = model_wiki[words_top_wiki[:1000]]
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, random_state=0)
words_top_wiki_tsne = tsne.fit_transform(words_top_vec_wiki)

In [97]:
p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=words_top_wiki_tsne[:,0],
                                    x2=words_top_wiki_tsne[:,1],
                                    names=words_top_wiki[:1000]))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)